In [3]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

# 1. Загрузка
df = pd.read_csv('weatherAUS.csv')

# 2. Очистка целевой переменной
# Нам нужно предсказывать RainToday. Удаляем строки, где это неизвестно.
df.dropna(subset=['RainToday'], inplace=True)

# 3. Отбор признаков (САМОЕ ВАЖНОЕ ДЛЯ ВАШЕЙ ЗАДАЧИ)
# Мы имитируем ситуацию "9 утра". Мы не знаем, что будет в 3 часа дня.

# Список колонок, которые мы удаляем, потому что это "будущее" или "ответ":
cols_to_drop = [
    'Date', 'Location',     # Локация может быть полезна, но для упрощения уберем
    'RainTomorrow',         # Это прогноз на завтра, нам не нужен
    'Rainfall',             # ВНИМАНИЕ: Это прямая подсказка (ответ), удаляем обязательно!
    'Evaporation', 'Sunshine', # Данные за весь день
    'MaxTemp',              # Максимум дня (еще не наступил)
    'RISK_MM',              # В некоторых версиях датасета есть этот столбец-спойлер
    
    # Удаляем все вечерние замеры (3pm), так как утром их у нас нет
    'WindGustSpeed', 'WindGustDir', # Порывы ветра обычно за день
    'WindDir3pm', 'WindSpeed3pm', 'Humidity3pm', 'Pressure3pm', 'Cloud3pm', 'Temp3pm'
]

# Удаляем только те, которые есть в датафрейме (чтобы не было ошибки, если каких-то нет)
actual_cols_to_drop = [c for c in cols_to_drop if c in df.columns]
df_morning = df.drop(columns=actual_cols_to_drop)

print("Признаки, на которых будем гадать (только утренние):")
print(df_morning.columns.tolist())

# 4. Предобработка (Пропуски и Кодирование)

# Целевая переменная (Y)
le = LabelEncoder()
df_morning['RainToday'] = le.fit_transform(df_morning['RainToday']) # No=0, Yes=1

# Разделяем на X и y
X = df_morning.drop('RainToday', axis=1)
y = df_morning['RainToday']

# Обработка категорий (Ветер в 9 утра)
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(include=['float64', 'int64']).columns

# Заполняем пропуски
imputer_num = SimpleImputer(strategy='median')
X[numerical_cols] = imputer_num.fit_transform(X[numerical_cols])

imputer_cat = SimpleImputer(strategy='most_frequent')
X[categorical_cols] = imputer_cat.fit_transform(X[categorical_cols])

# One-Hot Encoding для ветра (WindDir9am)
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# Нормализация
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# 5. Разделение
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 6. Обучение (Возьмем RandomForest, он хорошо работает с погодными условиями)
print("\nОбучаем модель предсказания дождя по утренним данным...")
model = RandomForestClassifier(n_estimators=100, max_depth=12, random_state=42, n_jobs=-1, class_weight='balanced')
model.fit(X_train, y_train)

# 7. Оценка
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print(f"ROC-AUC: {roc_auc_score(y_test, y_prob):.4f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Важность признаков (Что больше всего влияет на решение брать зонт?)
feature_importances = pd.DataFrame(model.feature_importances_,
                                   index = X.columns,
                                   columns=['importance']).sort_values('importance', ascending=False)

print("\nТоп-5 признаков, почему идет дождь:")
print(feature_importances.head(5))

# 8. Сериализация
joblib.dump(model, 'umbrella_model.pkl')
joblib.dump(scaler, 'umbrella_scaler.pkl')
# Также нужно сохранить список колонок (после get_dummies), чтобы приложение знало структуру
joblib.dump(X.columns.tolist(), 'model_columns.pkl')

print("\nМодель сохранена. Готова для приложения.")

Признаки, на которых будем гадать (только утренние):
['MinTemp', 'WindDir9am', 'WindSpeed9am', 'Humidity9am', 'Pressure9am', 'Cloud9am', 'Temp9am', 'RainToday']

Обучаем модель предсказания дождя по утренним данным...
ROC-AUC: 0.8516
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.80      0.85     22016
           1       0.51      0.74      0.61      6424

    accuracy                           0.78     28440
   macro avg       0.71      0.77      0.73     28440
weighted avg       0.82      0.78      0.80     28440


Топ-5 признаков, почему идет дождь:
             importance
Humidity9am    0.428248
Pressure9am    0.123313
Cloud9am       0.111332
MinTemp        0.104747
Temp9am        0.093515

Модель сохранена. Готова для приложения.
